In [ ]:
import glob

import matplotlib as mpl
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import skimage
from cytoolz import compose

# import cv2
from matplotlib.offsetbox import AnnotationBbox, DrawingArea, OffsetImage, TextArea
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_doh, blob_log

from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels
from paulssonlab.projects.molecule_counting.segmentation import invert, segment

mpl.rcParams["figure.figsize"] = (14, 14)

In [ ]:
# Lots of stuff commented out, this now takes in an image from Dirk's thesis (ClpX-BFP) as a test case.

In [ ]:
segment_phase = compose(segment, invert)

In [ ]:
filenames = glob.glob("/n/scratch3/users/j/jqs1/200228/*.nd2") + glob.glob(
    "/n/scratch3/users/j/jqs1/200305/*.nd2"
)
for i in range(len(filenames)):
    print(i, filenames[i])

In [ ]:
for file in filenames:
    try:
        print(file)
        nd2 = nd2reader.ND2Reader(file)
        print("Channels: ", nd2.metadata["channels"])
        # print('Fields of view: ', nd2.metadata['fields_of_view'])
        # print(nd2.metadata)
        print(nd2.sizes)
        fov_index = 0
        phase_image = nd2.get_frame_2D(v=0, c=0, z=fov_index)
        fluor_image = nd2.get_frame_2D(v=0, c=1, z=fov_index)
    #         fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
    #         ax = axes.ravel()
    #         ax[0].imshow(phase_image)
    #         ax[1].imshow(fluor_image, cmap="RdGy")
    except ValueError:
        print("ERROR IN", file)

In [ ]:
file = filenames[20]
print(file)
nd2 = nd2reader.ND2Reader(file)
print("Channels: ", nd2.metadata["channels"])
# print('Fields of view: ', nd2.metadata['fields_of_view'])
# print(nd2.metadata)
print(nd2.sizes)
fov_index = 0
phase_image = nd2.get_frame_2D(v=0, c=0, z=fov_index)
fluor_image = nd2.get_frame_2D(v=0, c=1, z=fov_index)
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_image)
ax[1].imshow(fluor_image, cmap="RdGy")

In [ ]:
# fluor_image = cv2.imread('test.png')
# print(np.shape(rgb2gray(fluor_image)))

# fluor_image = cv2.imread('test.png')
# fluor_image = cv2.cvtColor(fluor_image, cv2.COLOR_BGR2GRAY)
# plt.imshow(fluor_image, cmap = 'gray')

In [ ]:
# Crop image
xmin, xmax = 1500, 2200
ymin, ymax = 0, 800
phase_img = phase_image[ymin:ymax, xmin:xmax]
fluor_img = fluor_image[ymin:ymax, xmin:xmax]
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True)
ax = axes.ravel()
# ax[0].imshow(phase_img, cmap="gray")
# ax[1].imshow(fluor_img, cmap="seismic")

# Segment cells
phase_img = phase_image
fluor_img = fluor_image
seg = segment_phase(phase_img)
# ax[2].imshow(permute_labels(seg))
ax[2].imshow(seg)
print("Number of cells in fov: ", np.amax(seg))

In [ ]:
# Roughly segment cells, doesn't really matter, just so we have fluor_image and masked_fluor_image
# seg = np.ma.masked_where(fluor_img > 25, fluor_img)
# seg = seg.mask.astype(int)
# plt.imshow(seg)
# plt.show()

In [ ]:
def get_cells(seg):
    """takes in segmented image, returns a list of coordinates included in each cell."""
    number_of_cells = np.amax(seg)
    cell_labels = np.linspace(1, number_of_cells, number_of_cells)
    cells = []
    for cell in cell_labels:
        cell = np.where(seg == cell)
        cells.append(cell)
    return cells


cells = get_cells(seg)
print("Number of cells in fov: ", len(cells))

In [ ]:
def get_cell_boxes(seg):
    """takes in segmented image, returns coordinates of corners of each box with a cell."""
    cells = get_cells(seg)
    cell_boxes = []
    for cell in cells:
        cell_boxes.append(
            [np.amin(cell[0]), np.amax(cell[0]), np.amin(cell[1]), np.amax(cell[1])]
        )
    return cell_boxes


cell_label = 5
cell = cells[cell_label - 1]
boxes = get_cell_boxes(seg)
box = boxes[cell_label - 1]

In [ ]:
min_fluor = np.amin(fluor_img)
cells_fluor = np.ma.masked_where(seg == 0, fluor_img)
cells_fluor_max = np.amax(cells_fluor)
fluor_img_norm = (fluor_img - min_fluor) / (cells_fluor_max - min_fluor)

fig, axes = plt.subplots(10, 1, figsize=(5, 50))
ax = axes.ravel()
for i in range(10):
    cell = np.ma.masked_where(seg != i + 1, fluor_img_norm)
    ax[i].imshow(cell[boxes[i][0] : boxes[i][1], boxes[i][2] : boxes[i][3]])
    print(
        np.amin(cell),
        np.amax(cell),
        np.amax(cell) - np.amin(cell),
        np.std(cell),
        np.mean(cell),
    )

In [ ]:
# Get images (phase and fluor, and mask) of the cell.
phase_img_cell = phase_img[box[0] : box[1], box[2] : box[3]]
fluor_img_cell = fluor_img[box[0] : box[1], box[2] : box[3]]
seg_box = seg[box[0] : box[1], box[2] : box[3]]
grey_fluor_img_cell = rgb2gray(fluor_img_cell)

grey_fluor_img_cell_masked = np.ma.masked_where(
    seg_box != cell_label, grey_fluor_img_cell
)
phase_img_cell_masked = np.ma.masked_where(seg_box != cell_label, phase_img_cell)
fluor_img_cell_masked = np.ma.masked_where(seg_box != cell_label, fluor_img_cell)
fig, axes = plt.subplots(1, 6, figsize=(25, 10), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img_cell)
ax[0].set_title("Phase image")
ax[1].imshow(phase_img_cell_masked)
ax[1].set_title("Masked phase image")
ax[2].imshow(fluor_img_cell)
ax[2].set_title("Fluorescence image")
ax[3].imshow(fluor_img_cell_masked)
ax[3].set_title("Masked fluorescence image")
ax[4].imshow(grey_fluor_img_cell)
ax[4].set_title("Grayscale fluorescence image")
ax[5].imshow(grey_fluor_img_cell_masked)
ax[5].set_title("Masked grayscale fluorescence image")
plt.show()

In [ ]:
def mean_background_int(seg_img):
    """Takes in a segmented image and returns the mean intensity of the background (non cells)."""
    background = np.ma.masked_where(seg_img.mask == False, seg_img.data)
    return np.mean(background)


print(mean_background_int(grey_fluor_img_cell_masked))

In [ ]:
def std_background_int(seg_img):
    """Takes in a segmented image and returns the standard deviation of the background (non cells)
    intensity, so we can get an idea of the noise level."""
    background = np.ma.masked_where(seg_img.mask == False, seg_img.data)
    return np.std(background)


print(std_background_int(grey_fluor_img_cell_masked))

In [ ]:
mean_bg_int = mean_background_int(grey_fluor_img_cell_masked)
grey_fluor_img_cell_masked_bc = grey_fluor_img_cell_masked - mean_bg_int
plt.figure(figsize=(5, 5))
plt.imshow(grey_fluor_img_cell_masked_bc)
plt.show()

In [ ]:
def get_blobs_log(img):
    blobs_log = blob_log(img, max_sigma=10, threshold=1000)
    blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)
    return blobs_log


def get_blobs_dog(img):
    blobs_dog = blob_dog(img, max_sigma=10, threshold=10000)
    blobs_dog[:, 2] = blobs_dog[:, 2] * np.sqrt(2)
    return blobs_dog


def get_blobs_doh(img):
    return blob_doh(grey_fluor_img_cell_masked, max_sigma=10, threshold=0.01)


blobs_list = [
    get_blobs_log(grey_fluor_img_cell_masked_bc),
    get_blobs_dog(grey_fluor_img_cell_masked_bc),
    get_blobs_doh(grey_fluor_img_cell_masked_bc),
]
print(blobs_list)
colors = ["cyan", "lime", "red"]
titles = ["Laplacian of Gaussian", "Difference of Gaussian", "Determinant of Hessian"]
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(grey_fluor_img_cell_masked, cmap="gray")
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
        ax[idx].plot(x, y, "o", color=color)
#     ax[idx].set_axis_off()
plt.show()

In [ ]:
def get_blob_coordinates(blob_list):
    """Takes in lists of lists of blob coordinates and radii, as outputted by get_blobs_... and returns
    a list of lists of coordinates that are within those circles."""
    coordinates = []
    for blob in blob_list:
        y, x, r = blob
        ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
        rel_points = np.where(
            (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
        )
        points = [rel_points[0] + x - np.ceil(r), rel_points[1] + y - np.ceil(r)]
        coordinates.append(points)
    return coordinates


test = get_blob_coordinates(blobs_list[0])
plt.figure(figsize=(5, 5))
plt.imshow(grey_fluor_img_cell_masked_bc, cmap="gray")
for i in range(len(test)):
    plt.plot(*test[i], "r.")
plt.show()

In [ ]:
def total_fluor_ints_blobs(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the total fluorescence intensities
    within those blobs."""
    ints = []
    for i in range(len(blob_coordinates)):
        blob = blob_coordinates[i]
        ints.append(np.sum(img[list(map(int, blob[1])), list(map(int, blob[0]))]))
    return ints


print(
    total_fluor_ints_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def total_fluor_int_background(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the total fluorescence intensity
    outside the blobs. I.e. total fluorescence of the rest of the cell."""
    total_fluor = np.sum(img)
    blob_ints = total_fluor_ints_blobs(img, blob_coordinates)
    total_fluor -= np.sum(blob_ints)
    return total_fluor


print(
    total_fluor_int_background(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def fraction_int_in_blobs(img, blob_coordinates):
    total_fluor = np.sum(img)
    total_blob_ints = np.sum(total_fluor_ints_blobs(img, blob_coordinates))
    return total_blob_ints / total_fluor


print(
    fraction_int_in_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def mean_fluor_ints_blobs(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the mean fluorescence intensities
    within those blobs."""
    means = []
    for i in range(len(blob_coordinates)):
        blob = blob_coordinates[i]
        means.append(np.mean(img[list(map(int, blob[1])), list(map(int, blob[0]))]))
    return means


print(
    mean_fluor_ints_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def mean_fluor_int_background(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the mean fluorescence intensity
    outside the blobs. I.e. mean fluorescence of the rest of the cell."""
    bg_total_fluor = total_fluor_int_background(img, blob_coordinates)
    bg_points = np.ma.count(img)
    for blob in blob_coordinates:
        bg_points -= len(blob[0])
    return bg_total_fluor / bg_points


print(
    mean_fluor_int_background(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
print(np.std(grey_fluor_img_cell_masked_bc))

In [ ]:
# Dirk's data on the degradation assay.
WT_deg_rates = np.genfromtxt(
    "landgrafthesis_fig2_8_WT.csv", dtype="float", delimiter=","
)
WT_deg_low, WT_deg_high = (
    np.take(WTdeg_rates, 1, axis=1),
    np.take(WT_deg_rates, 0, axis=1),
)
WT_deg_rel = WT_deg_low / WT_deg_high

sfGFP_deg_rates = np.genfromtxt(
    "landgrafthesis_fig2_8_ClpPsfGFP.csv", dtype="float", delimiter=","
)
sfGFP_deg_low, sfGFP_deg_high = (
    np.take(sfGFP_deg_rates, 1, axis=1),
    np.take(sfGFP_deg_rates, 0, axis=1),
)
sfGFP_deg_rel = sfGFP_deg_low / sfGFP_deg_high

venus_deg_rates = np.genfromtxt(
    "landgrafthesis_fig2_8_ClpPvenus.csv", dtype="float", delimiter=","
)
venus_deg_low, venus_deg_high = (
    np.take(venus_deg_rates, 1, axis=1),
    np.take(venus_deg_rates, 0, axis=1),
)
venus_deg_rel = venus_deg_low / venus_deg_high

Dendra2_deg_rates = np.genfromtxt(
    "landgrafthesis_fig2_19_ClpPDendra2.csv", dtype="float", delimiter=","
)
Dendra2_deg_low, Dendra2_deg_high = (
    np.take(Dendra2_deg_rates, 1, axis=1),
    np.take(Dendra2_deg_rates, 0, axis=1),
)
Dendra2_deg_rel = Dendra2_deg_low / Dendra2_deg_high

Dronpa_deg_rates = np.genfromtxt(
    "landgrafthesis_fig2_19_ClpPDronpa.csv", dtype="float", delimiter=","
)
Dronpa_deg_low, Dronpa_deg_high = (
    np.take(Dronpa_deg_rates, 1, axis=1),
    np.take(Dronpa_deg_rates, 0, axis=1),
)
Dronpa_deg_rel = Dronpa_deg_low / Dronpa_deg_high

SNAP_deg_rates = np.genfromtxt(
    "landgrafthesis_fig2_19_ClpPSNAP.csv", dtype="float", delimiter=","
)
SNAP_deg_low, SNAP_deg_high = (
    np.take(SNAP_deg_rates, 1, axis=1),
    np.take(SNAP_deg_rates, 0, axis=1),
)
SNAP_deg_rel = SNAP_deg_low / SNAP_deg_high

# Upload the corresponding fluorescence images (don't have wt, only ClpX immunostaining):
sfGFP_img = cv2.imread("ClpP_sfGFP.png")
sfGFP_img = cv2.cvtColor(sfGFP_img, cv2.COLOR_BGR2GRAY)

venus_img = cv2.imread("ClpP_Venus.png")
venus_img = cv2.cvtColor(venus_img, cv2.COLOR_BGR2GRAY)

Dendra2_img = cv2.imread("ClpP_Dendra2.png")
Dendra2_img = cv2.cvtColor(Dendra2_img, cv2.COLOR_BGR2GRAY)

Dronpa_img = cv2.imread("ClpP_Dronpa.png")
Dronpa_img = cv2.cvtColor(Dronpa_img, cv2.COLOR_BGR2GRAY)

SNAP_img = cv2.imread("ClpP_SNAP.png")
SNAP_img = cv2.cvtColor(SNAP_img, cv2.COLOR_BGR2GRAY)

# Plot everything

fig, axes = plt.subplots(2, 3, figsize=(25, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].hist(WT_deg_rel, bins=np.linspace(0, 1, 50))
ax[0].set_title("ClpP-WT")

ax[1].hist(sfGFP_deg_rel, bins=np.linspace(0, 1, 50))
ax[1].set_title("ClpP-sfGFP")
imagebox = OffsetImage(sfGFP_img, zoom=0.25)
ab = AnnotationBbox(imagebox, (0.2, 10))
ax[1].add_artist(ab)

ax[2].hist(venus_deg_rel, bins=np.linspace(0, 1, 50))
ax[2].set_title("ClpP-Venus")
imagebox = OffsetImage(venus_img, zoom=0.25)
ab = AnnotationBbox(imagebox, (0.2, 10))
ax[2].add_artist(ab)

ax[3].hist(Dendra2_deg_rel, bins=np.linspace(0, 1, 50))
ax[3].set_title("ClpP-Dendra2")
imagebox = OffsetImage(Dendra2_img, zoom=0.25)
ab = AnnotationBbox(imagebox, (0.2, 10))
ax[3].add_artist(ab)

ax[4].hist(Dronpa_deg_rel, bins=np.linspace(0, 1, 50))
ax[4].set_title("ClpP-Dronpa")
imagebox = OffsetImage(Dronpa_img, zoom=0.25)
ab = AnnotationBbox(imagebox, (0.2, 10))
ax[4].add_artist(ab)

ax[5].hist(SNAP_deg_rel, bins=np.linspace(0, 1, 50))
ax[5].set_title("ClpP-SNAP")
imagebox = OffsetImage(SNAP_img, zoom=0.1)
ab = AnnotationBbox(imagebox, (0.2, 10))
ax[5].add_artist(ab)

plt.show()

In [ ]:
print("Mean relative degradation rates")
datasets = [
    WT_deg_rel,
    sfGFP_deg_rel,
    venus_deg_rel,
    Dendra2_deg_rel,
    Dronpa_deg_rel,
    SNAP_deg_rel,
]
datasets_str = [
    "WT_deg_rel",
    "sfGFP_deg_rel",
    "venus_deg_rel",
    "Dendra2_deg_rel",
    "Dronpa_deg_rel",
    "SNAP_deg_rel",
]
for i in range(len(datasets)):
    print(datasets_str[i] + ":")
    print("mean:", np.mean(datasets[i]))
    print("stdev:", np.std(datasets[i]))

In [ ]:
# However, might be better to do A/(A+B) and B/(A+B) instead of A/B, since this might be easier to fit distributions
# to, to try to model the expected distributions if there's one aggregate, or two aggregates etc.

WT_deg_frac_low = WT_deg_low / (WT_deg_low + WT_deg_high)
sfGFP_deg_frac_low = sfGFP_deg_low / (sfGFP_deg_low + sfGFP_deg_high)
venus_deg_frac_low = venus_deg_low / (venus_deg_low + venus_deg_high)
Dendra2_deg_frac_low = Dendra2_deg_low / (Dendra2_deg_low + Dendra2_deg_high)
Dronpa_deg_frac_low = Dronpa_deg_low / (Dronpa_deg_low + Dronpa_deg_high)
SNAP_deg_frac_low = SNAP_deg_low / (SNAP_deg_low + SNAP_deg_high)

WT_deg_frac_high = WT_deg_high / (WT_deg_low + WT_deg_high)
sfGFP_deg_frac_high = sfGFP_deg_high / (sfGFP_deg_low + sfGFP_deg_high)
venus_deg_frac_high = venus_deg_high / (venus_deg_low + venus_deg_high)
Dendra2_deg_frac_high = Dendra2_deg_high / (Dendra2_deg_low + Dendra2_deg_high)
Dronpa_deg_frac_high = Dronpa_deg_high / (Dronpa_deg_low + Dronpa_deg_high)
SNAP_deg_frac_high = SNAP_deg_high / (SNAP_deg_low + SNAP_deg_high)

# Plot everything

fig, axes = plt.subplots(2, 3, figsize=(25, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].hist(WT_deg_frac_low, bins=np.linspace(0, 1, 50))
ax[0].hist(WT_deg_frac_high, bins=np.linspace(0, 1, 50))
ax[0].set_title("ClpP-WT")

ax[1].hist(sfGFP_deg_frac_low, bins=np.linspace(0, 1, 50))
ax[1].hist(sfGFP_deg_frac_high, bins=np.linspace(0, 1, 50))
ax[1].set_title("ClpP-sfGFP")
imagebox = OffsetImage(sfGFP_img, zoom=0.3)
ab = AnnotationBbox(imagebox, (0.2, 25))
ax[1].add_artist(ab)

ax[2].hist(venus_deg_frac_low, bins=np.linspace(0, 1, 50))
ax[2].hist(venus_deg_frac_high, bins=np.linspace(0, 1, 50))
ax[2].set_title("ClpP-Venus")
imagebox = OffsetImage(venus_img, zoom=0.3)
ab = AnnotationBbox(imagebox, (0.2, 25))
ax[2].add_artist(ab)

ax[3].hist(Dendra2_deg_frac_low, bins=np.linspace(0, 1, 50))
ax[3].hist(Dendra2_deg_frac_high, bins=np.linspace(0, 1, 50))
ax[3].set_title("ClpP-Dendra2")
imagebox = OffsetImage(Dendra2_img, zoom=0.3)
ab = AnnotationBbox(imagebox, (0.2, 25))
ax[3].add_artist(ab)

ax[4].hist(Dronpa_deg_frac_low, bins=np.linspace(0, 1, 50))
ax[4].hist(Dronpa_deg_frac_high, bins=np.linspace(0, 1, 50))
ax[4].set_title("ClpP-Dronpa")
imagebox = OffsetImage(Dronpa_img, zoom=0.3)
ab = AnnotationBbox(imagebox, (0.2, 25))
ax[4].add_artist(ab)

ax[5].hist(SNAP_deg_frac_low, bins=np.linspace(0, 1, 50))
ax[5].hist(SNAP_deg_frac_high, bins=np.linspace(0, 1, 50))
ax[5].set_title("ClpP-SNAP")
imagebox = OffsetImage(SNAP_img, zoom=0.12)
ab = AnnotationBbox(imagebox, (0.2, 25))
ax[5].add_artist(ab)

plt.show()

In [ ]:
# Binomial distribution - each aggregate ends up in either A or B. Assume same probability for all.

In [ ]:
# Get standard deviations of images - need to roughly segment somehow.